In [1]:
!pip install tenseal


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.1 MB/s eta 0:00:00


In [3]:

from google.colab import files
import json
import numpy as np

# Upload the JSON file manually
uploaded = files.upload()  # Choose your MMU (1).json file when prompted
filename = list(uploaded.keys())[0]  # Get the actual filename

# Load the JSON data
with open(filename, "r") as f:
    data = json.load(f)

# Check data structure
print("Sample user IDs:", list(data.keys())[:3])
print("Sample feature shape:", np.array(data['1']['features']).shape)


Saving MMU (1).json to MMU (1) (1).json
Sample user IDs: ['1', '2', '3']
Sample feature shape: (10, 5)


In [4]:
import tenseal as ts

def create_ckks_context():
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]  # 128-bit security
    )
    context.generate_galois_keys()
    context.global_scale = 2 ** 40
    return context

# Create encryption context
context = create_ckks_context()
public_context = context.copy()
public_context.make_context_public()  # public copy for encryption only


In [5]:
# Encrypt all feature vectors and store them
encrypted_database = {}

for user_id, user_data in data.items():
    encrypted_features = []
    for vec in user_data["features"]:
        enc_vec = ts.ckks_vector(public_context, vec)
        encrypted_features.append(enc_vec.serialize())  # serialize to store
    encrypted_database[user_id] = encrypted_features


In [15]:
def cosine_similarity_encrypted(v1, v2, context):
    # Numerator: Encrypted dot product
    dot_product = v1.dot(v2)

    # Norms (plaintext)
    norm_v1 = np.sqrt(v1.dot(v1).decrypt()[0])
    norm_v2 = np.sqrt(v2.dot(v2).decrypt()[0])

    # Denominator (plaintext)
    denom = norm_v1 * norm_v2

    if denom == 0:
        return ts.ckks_vector(context, [0.0])  # prevent division by zero

    # Workaround: multiply with inverse
    inverse_denom = 1.0 / denom
    return dot_product * inverse_denom  # Element-wise encrypted * float


In [21]:
query_vec = data["1"]["features"][0]
enc_query = ts.ckks_vector(context, query_vec)

scores = []

for user_id, encrypted_feature_list in encrypted_database.items():
    for enc_feat_serialized in encrypted_feature_list:
        try:
            enc_feat = ts.ckks_vector_from(context, enc_feat_serialized)
            sim = cosine_similarity_encrypted(enc_query, enc_feat, context)
            score = sim.decrypt()[0]
            scores.append((user_id, score))
        except Exception as e:
            print(f"Error comparing with User {user_id}: {e}")


In [18]:
# Sort and display top matches
scores = sorted(scores, key=lambda x: x[1], reverse=True)

print("🔝 Top 5 Matches:")
for uid, sim in scores[:5]:
    print(f"User {uid} → Similarity: {sim:.4f}")

# Apply threshold
threshold = 0.95
best_match = scores[0]
if best_match[1] >= threshold:
    print(f"\n✅ Authenticated as User {best_match[0]} with similarity {best_match[1]:.4f}")
else:
    print(f"\n❌ Authentication failed. Best match is User {best_match[0]} with similarity {best_match[1]:.4f}")


🔝 Top 5 Matches:
User 1 → Similarity: 1.0000
User 42 → Similarity: 0.9941
User 6 → Similarity: 0.9860
User 11 → Similarity: 0.9856
User 27 → Similarity: 0.9855

✅ Authenticated as User 1 with similarity 1.0000


In [17]:
scores = sorted(scores, key=lambda x: x[1], reverse=True)
for user, score in scores[:10]:  # Top 10 matches
    print(f"User {user} - Similarity Score: {score:.4f}")


User 1 - Similarity Score: 1.0000
User 42 - Similarity Score: 0.9941
User 6 - Similarity Score: 0.9860
User 11 - Similarity Score: 0.9856
User 27 - Similarity Score: 0.9855
User 39 - Similarity Score: 0.9841
User 34 - Similarity Score: 0.9836
User 20 - Similarity Score: 0.9819
User 5 - Similarity Score: 0.9795
User 42 - Similarity Score: 0.9785


In [22]:
import numpy as np

# Get length of feature vector (from user "1")
N = len(data["1"]["features"][0])
print(f"Feature vector length: {N}")


Feature vector length: 5


In [23]:
# Generate a fake iris feature vector of the same length
fake_vec = np.random.rand(N).tolist()


In [24]:
# Encrypt the fake vector using the same CKKS context
enc_fake_query = ts.ckks_vector(context, fake_vec)


In [25]:
scores = []

for user_id, enc_feat_list in encrypted_database.items():
    for enc_feat_serialized in enc_feat_list:
        enc_feat = ts.ckks_vector_from(context, enc_feat_serialized)

        try:
            # Cosine similarity using workaround without scalar_div
            dot = enc_fake_query.dot(enc_feat)
            norm_fake = np.sqrt(enc_fake_query.dot(enc_fake_query).decrypt()[0])
            norm_db    = np.sqrt(enc_feat.dot(enc_feat).decrypt()[0])
            if norm_fake == 0 or norm_db == 0:
                similarity = 0.0
            else:
                similarity = dot * (1.0 / (norm_fake * norm_db))  # CKKSVector * float
            score = similarity.decrypt()[0]
            scores.append((user_id, score))
        except Exception as e:
            print(f"Error comparing with User {user_id}: {e}")


In [26]:
scores = sorted(scores, key=lambda x: x[1], reverse=True)

print("🔍 Top 5 Matches for Fake Iris:")
for uid, sim in scores[:5]:
    print(f"User {uid} → Similarity: {sim:.4f}")


🔍 Top 5 Matches for Fake Iris:
User 44 → Similarity: 0.8396
User 29 → Similarity: 0.7547
User 29 → Similarity: 0.7448
User 1 → Similarity: 0.7049
User 43 → Similarity: 0.6667


In [32]:
threshold = 0.90
best_match = scores[0]

if best_match[1] >= threshold:
    print(f"\n❌ FAKE iris accepted (this should not happen). Matched User {best_match[0]} with {best_match[1]:.4f}")
else:
    print(f"\n✅ FAKE iris correctly rejected. Highest similarity: {best_match[1]:.4f}")



✅ FAKE iris correctly rejected. Highest similarity: 0.8396
